In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#path's csv file, 
nov = pd.read_csv('/Users/tommy/Desktop/codici/archive/2019-Nov.csv', header='infer',
parse_dates=['event_time'],
date_parser=pd.to_datetime)

In [112]:
octo = pd.read_csv('/Users/tommy/Desktop/codici/archive/2019-Oct.csv', header='infer',
parse_dates=['event_time'],
date_parser=pd.to_datetime)

In [114]:
tot = pd.concat([nov,octo], axis = 0)

In [115]:
tot.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00+00:00,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00+00:00,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01+00:00,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01+00:00,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01+00:00,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [ ]:
#PURCHASE RATE: => #NUMBER OF PRODUCT SOLD/#USERS
subRQ6 = tot[['event_type','user_id','category_code','product_id']]

In [78]:
subRQ6.head()

,event_type,user_id,category_code,product_id
0,view,520088904,electronics,1003461
1,view,530496790,appliances,5000088
3,view,518085591,appliances,3601530
4,view,558856683,electronics,1004775
5,view,520772685,computers,1306894


In [83]:
# obtaining the number of product sold:
# by taking only rows about purchasing
# and at the end we count them (.shape[0])
# shape[0] are the rows of the dataset where the event is equal to purchase
# so it rapresents the product sold

df_prod_sold =  subRQ6[subRQ6['event_type']=='purchase']
prod_sold = df_prod_sold.shape[0]

In [84]:
prod_sold

682721

In [17]:
# in a similar way we calculate the number of user:
# this time we group by 'user_id' so we obtain the exact users's number
# we extract that number by the command shape[0] (number of rows)
users = subRQ6.groupby('user_id').product_id.count().shape[0]

In [80]:
users

3696117

In [85]:
#finally we calculate the actual purchase rate value
purchase_rate = prod_sold/users
round(purchase_rate,4)

0.1847

In [ ]:
#Now that we have the purchase rate we can calculate the conversion rate
# CONVERSION RATE = PURCHASE RATE/ #VIEWS

In [101]:
#same think of before
df_views = subRQ6[subRQ6['event_type']=='view']
views = df_views.shape[0]

In [39]:
#conversion rate calculated by following the previous formula
conversion_rate = purchase_rate / views
conversion_rate

3.903349387514857e-09

In [ ]:
#dropping nas values for both dataframe

In [102]:
df_views = df_views.dropna()

In [86]:
df_prod_sold = df_prod_sold.dropna()

In [87]:
#purchase rate of each category
df_prod_sold['category_code'] = [el.split('.')[0] for el in list(df_prod_sold['category_code']) if type(el) == str]


In [88]:
number_of_sales = df_prod_sold.groupby('category_code').product_id.count()

In [89]:
number_of_sales

category_code
accessories       2154
apparel          14215
appliances       99026
auto             10719
computers        34477
construction      8699
country_yard        60
electronics     493639
furniture        11542
kids              6166
medicine           344
sport             1489
stationery         191
Name: product_id, dtype: int64

In [65]:
# obtaining number of users for each categories
# first step: grouping for categories subRQ6
grouped_subRQ6 = subRQ6.groupby("category_code")

#now we want to extract the number of unique users for each category
user_of_each_cat = grouped_subRQ6.agg({"user_id": "nunique"})

In [79]:
user_of_each_cat.iloc[:,0]

category_code
accessories       71726
apparel          428720
appliances       710676
auto             164120
computers        363032
construction     137969
country_yard       4517
electronics     2061946
furniture        282369
kids             142084
medicine           4397
sport             46995
stationery         3691
Name: user_id, dtype: int64

In [76]:

#converting into a series
series_of_unique_users = user_of_each_cat.iloc[:,0]

In [100]:
#calculating the purchase rate => 
# number of product sold for each catogory / number of unique user for each category
purchase_rate_for_cat = number_of_sales.divide(series_of_unique_users)
round(purchase_rate_for_cat,4).sort_values(ascending=False)

category_code
electronics     0.2394
appliances      0.1393
computers       0.0950
medicine        0.0782
auto            0.0653
construction    0.0631
stationery      0.0517
kids            0.0434
furniture       0.0409
apparel         0.0332
sport           0.0317
accessories     0.0300
country_yard    0.0133
dtype: float64

In [ ]:
# now for the last step of this request 
# we calculate the conversion rate of each categories
# we have yet the purchase rate of product for each category
# in a similar way we are going to calculate the numer of views
# for each category and then we simply divide them.


In [103]:
df_views['category_code'] = [el.split('.')[0] for el in list(df_views['category_code']) if type(el) == str]

In [104]:
number_of_views = df_views.groupby('category_code').product_id.count()

In [105]:
number_of_views

category_code
accessories       380373
apparel          2942622
appliances       8037980
auto             1164278
computers        4029049
construction     1047072
country_yard       17299
electronics     21988192
furniture        2058798
kids              779517
medicine           20764
sport             240763
stationery         11943
Name: product_id, dtype: int64

In [111]:
coversion_rate_for_categories = purchase_rate_for_cat.divide(number_of_views)
coversion_rate_for_categories.sort_values(ascending=False)

category_code
stationery      4.332872e-06
medicine        3.767827e-06
country_yard    7.678567e-07
sport           1.315992e-07
accessories     7.895132e-08
construction    6.021591e-08
auto            5.609654e-08
kids            5.567148e-08
computers       2.357122e-08
furniture       1.985411e-08
appliances      1.733527e-08
apparel         1.126779e-08
electronics     1.088786e-08
dtype: float64

In [ ]:
#now we apply a traslation to plot better the results
coversion_for_plot = ((coversion_rate_for_categories)*10**8).sort_values(ascending=False)